In [1]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

from joblib import Parallel, delayed

import multiprocessing
from datetime import datetime

from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output

## Wybór urządzenia

In [2]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [3]:
torch.manual_seed(1010101011)
random.seed(1010101011)

In [4]:
class CharacterLSTM(nn.Module):
    def __init__(self, vocabsize, lstmlayers, hiddensize):
        super(CharacterLSTM, self).__init__()
        
        
        ## WARSTWY
        self.embd = nn.Embedding(vocabsize, vocabsize)
        self.LSTM1 = nn.GRU(vocabsize, hiddensize, lstmlayers, batch_first=True, bidirectional=True)
        self.linear_ins = nn.Linear(2*hiddensize, vocabsize)
#         self.linear_outs = nn.Linear(2*vocabsize, vocabsize)
        
        ## TRICKY LAYERS
#         self.norm = nn.BatchNorm1d(windowsize)
#         self.norm = nn.LayerNorm(2*vocabsize)
#         self.norm2 = nn.LayerNorm(vocabsize)
        self.drop = nn.Dropout(p=0.1)
        
        ## OUTS
        self.softmax = nn.LogSoftmax(dim=1)
        
        
#         self.init_weights()
        
    def init_weights(self):
        initrange = 0.1
        self.embd.weight.data.uniform_(-initrange, initrange)
        self.linear_ins.bias.data.zero_()
        self.linear_ins.weight.data.uniform_(-initrange, initrange)
#         self.linear_outs.bias.data.zero_()
#         self.linear_outs.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, x, hidden, NLL=True):
#         print(x.shape)
        
        # WEJSCIE
        y0 = self.embd(x)
#         y0  = self.drop(y0)
#         y0 = self.EmbeddingDrop(y0, p=0.2)

        
        # DROP
#         if drop>0:
#             y0 = self.EmbeddingDrop(y0, p=drop)
        
        # LSTM
        y, h1_ = self.LSTM1(y0, hidden)
        
        # TRICKY
#         y = self.norm(y)
        
        y = self.drop(y)
        
        # LINEAR OUT 1
        y = self.linear_ins(y)
#         y = self.norm(y)
#         y = self.linear_ins(y)
        
#         # TRICKY
#         y = self.norm2(y)
    
#         # TRICKY 2
#         y = self.drop(y)
        
        
#         # LINEAR OUT 2
#         y = F.relu(self.linear_outs(y))
      
#         y = self.norm2(y)

#         print(y, y.shape)
        
        if NLL:
            y = self.softmax(y[:,-1])
        
        # zwrot
        return y, h1_
    
#     def EmbeddingDrop(self, dataset, p=0.1):
#         w1, w2, w3 = dataset.shape
#         bern = torch.bernoulli((1-p) * torch.ones((w1, w2, 1))).to("cuda:0")
#         return dataset * bern

In [5]:
chartoidx = torch.load("../models/chartoidx.pth")
liczby = torch.load("../models/longestword.pth")

longestword = liczby[0]
lstms = 4
hiddensize = 40

chlstm = torch.load("../models/bezrelu_lstm_4_hidden_40_onecycle_1e-2-1e-8_rmsprop1e-3_700epoch_loss_2.683788537979126.pt", map_location=device).to(device)

In [6]:
t1 = torch.Tensor([[1,2,3], [4,5,6]])
t2 = torch.Tensor([9,9])

xx1 = t1.index_put_((torch.Tensor([0,1]).long(), torch.Tensor([1,1]).long()), t2)

print(t1.reshape(-1))

tensor([1., 9., 3., 4., 9., 6.])


## Działa dość wolno, ale nie ma już czasu na przyspieszane
Do przyspieszenia całe pętle for trzeba zamienić na operacje na macierzach, tak by jednocześnie sieć generowała hasła dla np. 1000 haseł

In [7]:
chardict = list(chartoidx.keys())
temperature = 0.50

# plikhasel = open("../haslaAI_bezrelu.txt", "a", encoding="utf8")

hiddens = torch.zeros(2*lstms, 1, hiddensize).to(device)

for _ in tqdm_notebook(range(10)):
    slowostart = chardict[random.randint(1, 61)]

    with torch.no_grad():
        lastchar = 0
        cnt = 0

        chlstm.eval()
        znaki = [chartoidx[item] for item in slowostart]
        bazastart = len(znaki)

        for __ in range(longestword - bazastart+1):
            znaki.append(chartoidx["<EMPTY>"])

        for item in range(longestword - bazastart+1):
            x = torch.Tensor(znaki).long().to(device).view(1, -1)

            out, _ = chlstm(x, hiddens, NLL=False)
            
            zwrot = out[:,-1].view(-1)
            exped = zwrot.data.div(temperature).exp()
            top_i = torch.multinomial(exped, 1)
    #         softmaxed = torch.softmax(top_i, 0)
    #         charid = torch.argmax(softmaxed, 0)
            charid = top_i[0]

            znaki[item+bazastart] = charid


        slowo = ""
        for item in znaki:
            if item != 0:
                slowo+=chardict[item]

        print(slowo)
#         plikhasel.write(slowo+"\n")
# plikhasel.close()

048320003
farararn210
hareselk2
Gkanie15
aronder1
bala112259
80299895
90017016011
Kora11121010
onia09619

